# Infer LIPID BODIES - part 9️⃣

--------------

## OBJECTIVE: ✅ Infer sub-cellular component LIPID BODIES  in order to understand interactome 



Dependencies:
The ER  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice, edge_preserving_smoothing_3d )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI
from infer_subc_2d.organelles.soma import infer_SOMA
from infer_subc_2d.organelles.cytosol import infer_CYTOSOL
from infer_subc_2d.constants import TEST_IMG_N

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

# IMAGE PROCESSING Objective 9️⃣:  infer LIPID BODIES 
> Back to  [OUTLINE: Objectives 9️⃣](00_pipeline_setup.ipynb#summary-of-objectives)

## summary of steps (Workflow #1 & #2)

INPUT
- ch 7
- CY mask

PRE-PROCESSING
- rescale 
-  median, window 2
-  
CORE-PROCESSING
   - "robust background" 2 STD

POST-PROCESSING
  - n/a

OUTPUT
- object BD


NOTE:  consider using [Fibrillarin](https://www.allencell.org/cell-observations/category/fibrillarin) (nucleous marker) workflow from AllenCell.  playground_spotty.ipynb  or maybe [centrin](https://www.allencell.org/cell-observations/category/centrin)





------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [3]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [4]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [5]:


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


## get the inferred soma & cyto objects

(takes < 1 sec)

In [6]:
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, default_params) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


# WORKFLOW #1 

Not a good thing to follow from  the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.






## summary of steps


INPUT
- ch 4
- CY mask

PRE-PROCESSING
- edge preserving scaling

CORE-PROCESSING
- vesselness slice-by-slice

POST-PROCESSING
  - S  - remove objects less than 2x2 pixels (area = 4)

OUTPUT
- object LIPID BODIES 





### INPUT

In [7]:

###################
# INPUT
###################
raw_lipid   = img_2D[6].copy()
# mask object
masked_lipid = apply_mask(raw_lipid,CY_object)


### PRE-PROCESSING

In [8]:
###################
# PRE_PROCESSING
###################


intensity_norm_param = [0]  # CHECK THIS
# Linear-ish smoothing
struct_img = intensity_normalization( raw_lipid ,  scaling_param=intensity_norm_param)

med_filter_size = 2   
structure_img_median = ndi.median_filter(struct_img, size=med_filter_size)

gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0

structure_img_smooth = ndi.gaussian_filter( struct_img, 
                                                                            sigma=gaussian_smoothing_sigma, 
                                                                            mode="nearest", 
                                                                            truncate=gaussian_smoothing_truncate_range)


intensity normalization: min-max normalization with NO absoluteintensity upper bound


### CORE PROCESSING

In [15]:
###################
# CORE_PROCESSING
###################


threshold_val = threshold_li(structure_img_smooth)

threshold_factor = 0.99 #from cellProfiler
thresh_min = 0.5
thresh_max = 1.0
threshold = min( max(threshold_val*threshold_factor, thresh_min), thresh_max)


li_thresholded = structure_img_smooth > threshold

threshold

0.4

### POST-PROCESSING

In [16]:

###################
# POST_PROCESSING
###################
hole_width = 2.5  
# # wrapper to remoce_small_objects
removed_holes = remove_small_holes(li_thresholded, hole_width ** 2 )

small_object_max = 4

cleaned_img = size_filter_2D(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max ** 2, 
                                                         connectivity=1)


LD_object = cleaned_img
LD_labels = label(cleaned_img   )
LD_signal = struct_img


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [17]:


viewer = napari.view_image(
    structure_img_smooth,
    scale=scale
)


viewer.scale_bar.visible = True
viewer.add_image(
    LD_object,
    scale=scale
)

viewer.add_labels(
    LD_labels,
    scale=scale
)


<Labels layer 'LD_labels' at 0x187d4b3d0>

In [18]:

viewer.add_image(
     apply_mask(img_2D[7].copy(),CY_object),
    scale=scale
)

viewer.add_image(
     img_2D[6].copy(),
    scale=scale
)


<Image layer 'Image [1]' at 0x18aec3070>

### DEFINE `def infer_LIPID_DROPLET(struct_img, out_path, cyto_labels, in_params):
` function

In [25]:
##########################
#  infer_LIPID_DROPLET
##########################
def _infer_LIPID_DROPLET(struct_img, CY_object,  in_params) -> tuple:
    """
    Procedure to infer LIPID_DROPLET  from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 2d (actually single Z) image containing the LIPID_DROPLET signal

    CY_object: np.ndarray boolean
        a 2d (1Z- 3D) image containing the CYTO 

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of LD
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()
    
    struct_img = apply_mask(struct_img,CY_object)

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    med_filter_size = 2   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = ndi.median_filter( struct_img,
                                                         size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = ndi.gaussian_filter( struct_img,
                                                            sigma=gaussian_smoothing_sigma, 
                                                            mode="nearest", 
                                                            truncate=gaussian_smoothing_truncate_range)
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    ###################
    # CORE_PROCESSING
    ###################
    threshold_val = threshold_li(struct_img)

    threshold_factor = 0.99 #from cellProfiler
    thresh_min = .5
    thresh_max = 1.
    threshold = min( max(threshold_val*threshold_factor, thresh_min), thresh_max)
    out_p['threshold_factor'] = threshold_factor
    out_p['thresh_min'] = thresh_min
    out_p['thresh_max'] = thresh_max

    struct_obj = struct_img > threshold

    ###################
    # POST_PROCESSING
    ###################
    hole_width = 2.5  
    # # wrapper to remoce_small_objects
    struct_obj = remove_small_holes(struct_obj, hole_width ** 2 )
    out_p['hole_width'] = hole_width

    small_object_max = 4
    struct_obj = size_filter_2D(struct_obj, 
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    retval = (struct_obj, out_p)
    return retval

# TEST  `_infer_LIPID_DROPLET` function

In [26]:

###################
# INPUT
###################
raw_lipid   = img_2D[6].copy()

LD_object, out_p =  _infer_LIPID_DROPLET(raw_lipid, CY_object, default_params) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [30]:

viewer.add_image(
    LD_object,
    scale=scale
)
viewer.add_labels(
    label(LD_object),
    scale=scale
)

<Labels layer 'Labels [2]' at 0x18a895c40>

In [29]:
from infer_subc_2d.organelles.lipid import infer_LIPID_DROPLET


img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_lipid   = img_2D[6].copy()

LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, default_params) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [ ]:
# save updated parameters for ongoing testing
save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )

'/Users/ahenrie/Projects/Imaging/data/intermediate/ZSTACK_PBTOhNGN2hiPSCs_BR3_N14_Unmixed.czi_params.pkl'